images - line by line, cmaera - col by col
    Q: is this hardware specific?
screen waits 1 frame cycle, camera waits 1 frame cycle 
    Q: r they the same?


two kinds of challenges:
    background challenge: one color
    lighting challenge: belt of different colr from background color
        belt is "lighting area"

ROI: region that camera is scanning when the screen is displaying the lighting area

In [4]:
import numpy as np
import cv2 
from sklearn.linear_model import LinearRegression as lr 

In [ ]:
def eqn_2(pixel, color1, color2, E):
    #INPUTS:
        #pixel: a single pixel with all 3 color channels
        #color1: background color being shown on screen
        #color2: primary color being shown on screen (band)
        #E: illuminance for all 3 channels 
    #confirm that I{c1}/I{c2} = E{c1}/E{c2}
    #(where c1 and c2 are the 2 colors being shown on the screen)  

    iFraction = pixel[color1]/pixel[color2]
    eFraction = E[color1]/E[color2]
    epsilon = 0.5
    return iFraction- eFraction <= epsilon  


def roi(frames, t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E):
    # STEP 1: Calculate ROI
    # calculate start time to show the lighting area, t_u
    t_u = t_begin + (u / rows) * t_frame
    # find image - frame at t_u
    image = frames[fps * t_u]
    # calculate shift, l
    l = cols * (t_u - ct_k) / (ct_frame)
    

    # Apply Eqn 2 on every pixel between response of lighting challenge and background challenge
    for r in image.shape[0]:
        for c in image.shape[1]:
            consistent = eqn_2(image[r][c][:], color1, color2, E)
            if not consistent:
                print("Not consistent!")
                return
    
 
    # regression model
    #image is 100 columns, ROI is columns 40-65
    a,b=10,50
    ROI = (a,b)
    
    imgWidth = image.shape[1]
    y_hat = 0
    if a < 0.25*imgWidth:
        avgCols = None # take average value of columns [a,25]
        output = None #lr1.predict(avgCols) #run regression on model 1
        y_hat += (0.25*imgWidth-a)*output
    if b < 0.5*imgWidth:
        #model2: avg. value of columns [25, b]
        # run regression on model 2 
        avgCols = None
        output = None #lr2.predict(avgCols)
        y_hat += (b-0.25*imgWidth)*output

    if b < 0.75*imgWidth and a > 0.25*imgWidth:
        #model 3: avg value of columns [50,b]
        avgCols = None
        output = None #lr3.predict(avgCols)
        y_hat += (b-0.5*imgWidth)*output

    if b >=0.75*imgWidth:
        #model 4 : avg value of columns [75,b]
        avgCols = None
        output = None #lr4.predict(avgCols)
        y_hat += (b-0.75*imgWidth)*output
    
    y_hat = y_hat /imgWidth

    return y_hat


def criteria(frames):
    t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E = 1,2,3,4,5,6,7,8,9,10,11
    
    dVals = []
    for frame in frames:
        y_hat_i = roi(frames, t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E)

        # calc criteria
        d = 0.25 #% of screen the band is shown on 
        imgRows = frame.shape[0] 
        
        d_i = y_hat_i - (u + u+imgRows*0.25)/2
        dVals.append(d_i)

    mean = np.mean(dVals)
    var = np.var(dVals)

    threshold = -5
    return mean * np.sqrt(var) < np.exp(threshold)


IMPLEMENTATION

In [ ]:
## 1. PASSING EQN2 (ratio of illuminance is met) 
img = cv2.imread('')
background, primary = 0,1 #0=B, 1=G, 2=R
illuminance = [255,255,0] #values of illuminance shown on screen

for r in range(img.shape[0]):
    for c in range(img.shape[1]):
        eqn_2(img[c][r][:], background, primary, illuminance)
#for each pixel in the img: 
    #get the relevant color
    #input into eqn_2

In [ ]:
#training linear regression models to be used to identify location of band of color
#TO DO: need data to train on !! 
lr1 = lr().fit(x1_train, y1_train)
lr2 = lr().fit(x2_train, y2_train)
lr3 = lr().fit(x3_train, y3_train)
lr4 = lr().fit(x4_train, y4_train)
